## Introduction

Data scientists commonly spend over half their time cleaning data, so knowing how to clean "messy" data is an extremely important skill.

### Reading CSV Files with Encodings

We'll learn the basics of data cleaning with pandas as we work with `laptops.csv`, a CSV file containing information about 1,300 laptop computers.

Computers, at their lowest levels, can only understand binary - 0 and 1- and encodings are systems for representing characters in binary.
Something we can do if our file has an unknown encoding is to try the most common encodings:
- UTF-8
- Latin-1 (also known as ISO-8859-1)
- Windows-1251

In [1]:
import pandas as pd

laptops = pd.read_csv("data/laptops.csv", encoding='Latin-1')
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 66.2+ KB


### Cleaning Column Names

We can see that every column is represented as the `object` type, indicating that they are represented by strings, not numbers. Also, one of the columns, `Operating System Version`, has null values.

The column labels have a variety of upper and lowercase letters, as well as spaces and parentheses, which will make them harder to work with and read. One noticeable issue is that the `" Storage"` column name has a space in front of it. These quirks with column labels can sometimes be hard to spot, so removing extra whitespaces from all column names will save us more work in the long run.

We can access the column axis of a dataframe using the `DataFrame.columns` attribute. This returns an index object — a special type of NumPy ndarray — with the labels of each column

In [2]:
new_columns = []

for col in laptops.columns:
    column = col.strip() # Remove whitespaces
    new_columns.append(column)
    
laptops.columns = new_columns # Assign clean column names to the original DF

In [3]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
Storage                     1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 66.2+ KB


The column labels still have a variety of upper and lowercase letters, as well as parentheses, which will make them harder to work with and read. Let's finish cleaning our column labels by:
- Replacing spaces with underscores.
- Removing special characters.
- Making all labels lowercase.
- Shortening any long column names.

In [4]:
def clean_col(col_name):
    col = col_name.strip() # Remove whitespaces
    col = col.replace("Operating System", "os") # Replace substring
    col = col.replace(" ", "_") # Replace spaces with underscores
    # Replace parenthesis
    col = col.replace("(", "") 
    col = col.replace(")", "")
    col = col.lower()
    return col

new_columns = []

for column in laptops.columns:
    column_name = clean_col(column)
    new_columns.append(column_name)
    
laptops.columns = new_columns

In [5]:
laptops.columns

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')

### Converting String Columns to Numeric

We observed earlier that all 13 columns have the object dtype, meaning they're stored as strings. Let's look at the first few rows of some of our columns

In [6]:
laptops.iloc[:5, 2:5]

,category,screen_size,screen
0,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600
1,Ultrabook,"13.3""",1440x900
2,Notebook,"15.6""",Full HD 1920x1080
3,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800
4,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600


Of these three columns, we have three different types of text data:
- `category`: Purely text data - there are no numeric values.
- `screen_size`: Numeric data stored as text data because of the " character.
- `screen`: A combination of pure text data with numeric data.

Because the values in the screen_size column are stored as text data, we can't sort them numerically. For instance, if we wanted to select laptops with screens 15" or larger, we'd be unable to do so.

Whenever we convert text to numeric data, we can follow this data cleaning workflow:
<img src="_images/cleaning_workflow.svg" />

The first step is to __explore the data__. One of the best ways to do this is to use the `Series.unique()` method to view all of the unique values in the column

In [7]:
laptops["screen_size"].dtype

dtype('O')

In [8]:
laptops["screen_size"].unique()

array(['13.3"', '15.6"', '15.4"', '14.0"', '12.0"', '11.6"', '17.3"',
       '10.1"', '13.5"', '12.5"', '13.0"', '18.4"', '13.9"', '12.3"',
       '17.0"', '15.0"', '14.1"', '11.3"'], dtype=object)

The next step is to __identify patterns and special cases__.
- All values in this column follow the same pattern - a series of digit and period characters, followed by a quote character (").
- There are no special cases. Every value matches the same pattern.
- We'll need to convert the column to a float dtype, as the int dtype won't be able to store the decimal values.

Let's use the second step to study the `ram` column:

In [9]:
unique_ram = laptops["ram"].unique()

### Removing non-digit characters

To convert both `screen_size` and `ram` column to numeric dtypes, we have to __remove non-digit characters__.

The pandas library contains dozens of vectorized string methods we can use to manipulate text data, many of which perform the same operations as Python string methods. Most vectorized string methods are available using the `Series.str` accessor, which means we can access them by adding `str` between the series name and the method name.

We can use the `Series.str.replace()` method, which is a vectorized version of the Python `str.replace()` method to remove all the quote characters from every string in the `screen_size` column.

In [10]:
laptops["screen_size"] = laptops['screen_size'].str.replace('"', '')
laptops["screen_size"].unique()

array(['13.3', '15.6', '15.4', '14.0', '12.0', '11.6', '17.3', '10.1',
       '13.5', '12.5', '13.0', '18.4', '13.9', '12.3', '17.0', '15.0',
       '14.1', '11.3'], dtype=object)

Let's repeat this step for the `ram` column

In [11]:
laptops["ram"] = laptops["ram"].str.replace('GB', '')
unique_ram = laptops["ram"].unique()

In [12]:
unique_ram

array(['8', '16', '4', '2', '12', '6', '32', '24', '64'], dtype=object)

### Converting Columns to Numeric Dtypes

Now, we can __convert (or cast) the columns to a numeric dtype__.

To do this, we use the `Series.astype()` method. To convert the column to a numeric dtype, we can use either `int` or `float` as the parameter for the method. Since the `int` dtype can't store decimal values, we'll convert the `screen_size` column to the `float` dtype.

In [13]:
laptops["screen_size"] = laptops["screen_size"].astype(float)
laptops["screen_size"].unique()

array([13.3, 15.6, 15.4, 14. , 12. , 11.6, 17.3, 10.1, 13.5, 12.5, 13. ,
       18.4, 13.9, 12.3, 17. , 15. , 14.1, 11.3])

In [14]:
import numpy as np
laptops["ram"] = laptops["ram"].astype(np.int64)
dtypes = laptops.dtypes
dtypes

manufacturer     object
model_name       object
category         object
screen_size     float64
screen           object
cpu              object
ram               int64
storage          object
gpu              object
os               object
os_version       object
weight           object
price_euros      object
dtype: object

In [15]:
laptops["ram"].unique()

array([ 8, 16,  4,  2, 12,  6, 32, 24, 64], dtype=int64)

### Renaming Columns

Now that we've converted our columns to numeric dtypes, the final step is to __rename the column__.

To stop us from losing information the helps us understand the data, we can use the `DataFrame.rename()` method to rename the column from `screen_size` to `screen_size_inches`

Below, we specify the `axis=1` parameter so pandas knows that we want to rename labels in the column axis

Note that we can either use `inplace=True` or assign the result back to the dataframe - both will give us the same results.

In [16]:
laptops.rename({"screen_size" : "screen_size_inches"}, axis=1, inplace=True)
laptops.rename({"ram" : "ram_gb"}, axis=1, inplace=True)
ram_gb_desc = laptops["ram_gb"].describe()
ram_gb_desc

count    1303.000000
mean        8.382195
std         5.084665
min         2.000000
25%         4.000000
50%         8.000000
75%         8.000000
max        64.000000
Name: ram_gb, dtype: float64

### Extracting Values from Strings

Sometimes, it can be useful to extract non-numeric values from within strings.

In [17]:
laptops["gpu"].head()

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
3              AMD Radeon Pro 455
4    Intel Iris Plus Graphics 650
Name: gpu, dtype: object

The information in this column seems to be a manufacturer (Intel, AMD) followed by a model name/number. Let's extract the manufacturer by itself so we can find the most common ones.

Because each manufacturer is followed by a whitespace character, we can use the `Series.str.split()` method to extract this data.

In [18]:
laptops["gpu"].head().str.split()

0    [Intel, Iris, Plus, Graphics, 640]
1           [Intel, HD, Graphics, 6000]
2            [Intel, HD, Graphics, 620]
3               [AMD, Radeon, Pro, 455]
4    [Intel, Iris, Plus, Graphics, 650]
Name: gpu, dtype: object

This method splits each string on the whitespace; the result is a series containing individual Python lists.

Just like with lists and ndarrays, we can use bracket notation to access the elements in each list in the series. With series, however, we use the `str` accessor followed by `[]` (brackets)

In [19]:
laptops["gpu_manufacturer"] = laptops["gpu"].str.split().str[0] # add the GPU manufacturer info from the 'gpu' column into a new column
laptops["cpu_manufacturer"] = laptops["cpu"].str.split().str[0]
cpu_manufacturer_counts = laptops["cpu_manufacturer"].value_counts()

### Correcting Bad Values

If your data has been scraped from a webpage or if there was manual data entry involved at some point, you may end up with inconsistent values. Let's look at an example from our os column:

In [20]:
laptops["os"].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64

We see two versions of the Apple OS - `Mac OS` and `macOS`. One way we can fix this is with the `Series.map()` method.
- The `Series.map()` method is ideal when we want to change multiple values in a column, but we'll use it now as an opportunity to learn how the method works.
- The most common way to use `Series.map()` is with a dictionary.
- One important thing to remember with `Series.map()` is that if a value from your series doesn't exist as a key in your dictionary, it will convert that value to `NaN`.

In [21]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops["os"] = laptops["os"].map(mapping_dict)

In [22]:
laptops["os"].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64

### Dropping Missing Values

In pandas, null values will be indicated by either `NaN` or `None`.

Recall that we can use the `DataFrame.isnull()` method to identify missing values, which returns a boolean dataframe. We can then use the `DataFrame.sum()` method to give us a count of the `True` values for each column

In [23]:
laptops.isnull().sum()

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64

It's now clear that we have only one column with null values, `os_version`, which has 170 missing values.

There are a few options for handling missing values:
- Remove any rows that have missing values.
- Remove any columns that have missing values.
- Fill the missing values with some other value.
- Leave the missing values as is.

The first two options are often used to prepare data for machine learning algorithms, which are unable to be used with data that includes null values.

We can use the `DataFrame.dropna()` method to remove or __drop__ rows and columns with null values. The `DataFrame.dropna()` method accepts an `axis` parameter, which indicates whether we want to drop along the column or index axis.

The default value for the axis parameter is 0, so `df.dropna()` returns an identical result to `df.dropna(axis=0)`

In [24]:
laptops_no_null_rows = laptops.dropna()
laptops_no_null_cols = laptops.dropna(axis=1)

### Filling Missing Values

While dropping rows or columns is the easiest approach to deal with missing values, it may not always be the best approach. For example, removing a disproportionate amount of one manufacturer's laptops could change our analysis.

Because of this, it's a good idea to explore the missing values in the os_version column before making a decision. We can use Series.value_counts() to explore all of the values in the column, but we'll use a parameter we haven't seen before.

In [25]:
laptops["os_version"].value_counts(dropna=False)

10      1072
NaN      170
7         45
10 S       8
X          8
Name: os_version, dtype: int64

Because we set the `dropna` parameter to `False`, the result includes `null` values. We can see that the majority of values in the column are `10` and missing values are the next most common.

Let's also explore the `os` column, since it's is closely related to the `os_version` column. We'll only look at rows in which the `os_version` is missing.

In [26]:
os_with_null_v = laptops.loc[laptops["os_version"].isnull(), "os"]
os_with_null_v.value_counts()

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64

Immediately, we can observe a few things:
- The most frequent value is "No OS". This is important to note because if there is no os, there _shouldn't_ be a version defined in the `os_version` column.
- Thirteen of the laptops that come with macOS do not specify the version. We can use our knowledge of MacOS to confirm that `os_version` should be equal to `X`.

In [28]:
laptops.loc[laptops["os"] == 'macOS', "os_version"] = 'X'

In [30]:
laptops["os_version"].value_counts(dropna=False)

10      1072
NaN      157
7         45
X         21
10 S       8
Name: os_version, dtype: int64

In [31]:
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"

value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

In [32]:
value_counts_after

Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64

We have succesfully filled the missing values for the `macOS` and `No OS` OS Versions

## Challenge: Clean a String Column

In this challenge, we'll clean the `weight` column.

While it appears that the `weight` column may just need the `kg` characters removed from the end of each string, there is one special case - one of the values ends with `kgs`, so you'll have to remove both `kg` and `kgs` characters.

In the last step of this challenge, we'll also ask you to use the `DataFrame.to_csv()` method to save the cleaned data to a CSV file. It's a good idea to save a CSV when you finish cleaning in case you wish to do analysis later.

In [33]:
laptops["weight"].head()

0    1.37kg
1    1.34kg
2    1.86kg
3    1.83kg
4    1.37kg
Name: weight, dtype: object

In [34]:
laptops["weight"] = laptops["weight"].str.replace("kg","")
laptops["weight"] = laptops["weight"].str.replace("s", "")
laptops["weight"] = laptops["weight"].astype(float)
laptops.rename({"weight" : "weight_kg"}, axis=1, inplace=True)
# laptops.to_csv("data/laptops_cleaned.csv", index=False)

### Next Steps:

Our data set is ready for some analysis, but there are still some data cleaning tasks left! Here are some of the next steps:
- Convert the price_euros column to a numeric dtype.
- Extract the screen resolution from the screen column.
- Extract the processor speed from the cpu column.

### Extract Screen Resolution: 

In [35]:
laptops["screen"].head().str.split()

0    [IPS, Panel, Retina, Display, 2560x1600]
1                                  [1440x900]
2                       [Full, HD, 1920x1080]
3    [IPS, Panel, Retina, Display, 2880x1800]
4    [IPS, Panel, Retina, Display, 2560x1600]
Name: screen, dtype: object

In [36]:
laptops["screen_resolution"] = laptops["screen"].str.split().str[-1]
laptops["screen_resolution"].head()

0    2560x1600
1     1440x900
2    1920x1080
3    2880x1800
4    2560x1600
Name: screen_resolution, dtype: object

### Extract processor speed from the cpu column:

In [37]:
laptops["cpu"].head().str.split()

0           [Intel, Core, i5, 2.3GHz]
1           [Intel, Core, i5, 1.8GHz]
2    [Intel, Core, i5, 7200U, 2.5GHz]
3           [Intel, Core, i7, 2.7GHz]
4           [Intel, Core, i5, 3.1GHz]
Name: cpu, dtype: object

In [39]:
laptops["processor_speed"] = laptops["cpu"].str.split().str[-1]
laptops["processor_speed"].dtypes

dtype('O')

In [40]:
laptops["processor_speed"] = laptops["processor_speed"].str.replace("GHz","")
laptops["processor_speed"] = laptops["processor_speed"].astype(float)
laptops.rename({"processor_speed" : "processor_GHz"}, axis=1, inplace=True)

### Convert the price_euros column to a numeric dtype:

In [41]:
laptops["price_euros"].head()

0    1339,69
1     898,94
2     575,00
3    2537,45
4    1803,60
Name: price_euros, dtype: object

In [42]:
laptops["price_euros"] = laptops["price_euros"].str.replace(",", ".")
laptops["price_euros"] = laptops["price_euros"].astype(float)
laptops["price_euros"].head()

0    1339.69
1     898.94
2     575.00
3    2537.45
4    1803.60
Name: price_euros, dtype: float64

In [43]:
# Save to a new CSV file
laptops.to_csv("data/laptops_cleaned.csv", index=False)

Here are some questions you might like to answer in your own time by analyzing the cleaned data:
- Are laptops made by Apple more expensive than those made by other manufacturers?
- What is the best value laptop with a screen size of 15" or more?
- Which laptop has the most storage space?